In [66]:
import utils
import predict
import time as tm
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.datasets import make_classification
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split

In [67]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.utils.data as data_utils
import torch.nn.functional as F

In [68]:
# load Data
dictSize = 225
(X, y) = utils.loadData( "train", dictSize = dictSize )
X = scipy.sparse.csr_matrix.toarray(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [69]:
# https://discuss.pytorch.org/t/multi-class-classification/47565
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        return out
    
    def predict(self, x):
        with torch.no_grad():
            outp = self.forward(x)
            return F.softmax(outp)
# add softmax layer: used for multiclass classification

In [81]:
# model declared
model = NeuralNet(225 , 10000, 51)  
# earlier hidden layers- 1000 
# 1 - poor, 2- ok

In [82]:
loss_function = nn.CrossEntropyLoss()
model_opt = optim.SGD(model.parameters(), lr = 0.02)

train_set_X = Variable(torch.from_numpy(X_train)).float()
train_set_y = Variable(torch.LongTensor(y_train)).long()
test_set_X = Variable(torch.from_numpy(X_test)).float()
test_set_y = Variable(torch.LongTensor(y_test)).long()

In [83]:
# 1
epochs = 50
for epochs in range(epochs):
    model_opt.zero_grad()
    out = model(train_set_X)
    loss = loss_function(out, train_set_y)
    loss.backward()
    model_opt.step()

In [84]:
ans = model.predict(test_set_X)
# for i in range(3299):
#     ans_max,ind = torch.max(ans[i],0)
#     print (test_set_y[i].numpy() - ind.numpy())
    

/home/aditya/Vpy35/lib/python3.5/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [85]:
k = 5
y_pr = ans.numpy() 
y_pred = np.argsort(-y_pr,axis=1)[:,:k]
print (y_pred)
print (y_test)

[[ 3  4  1  7  2]
 [ 2  1  4  3  9]
 [ 3  1  4  2 10]
 ...
 [ 1  2  4  9  3]
 [ 2  1  4  3  9]
 [ 3  4  1  2 10]]
[3. 2. 3. ... 9. 4. 3.]


In [86]:
# eval functions for Deep Learning
preck = utils.getPrecAtK( y_test, y_pred, k )
# The macro precision code takes a bit longer to execute due to the for loop over labels
mpreck = utils.getMPrecAtK( y_test, y_pred, k )

# According to our definitions, both prec@k and mprec@k should go up as k goes up i.e. for your
# method, prec@i > prec@j if i > j and mprec@i > mprec@j if i > j. See the assignment description
# to convince yourself why this must be the case.

print( "prec@1: %0.3f" % preck[0], "prec@3: %0.3f" % preck[2], "prec@5: %0.3f" % preck[4] )
# Dont be surprised if mprec is small -- it is hard to do well on rare error classes
print( "mprec@1: %0.3e" % mpreck[0], "mprec@3: %0.3e" % mpreck[2], "mprec@5: %0.3e" % mpreck[4] )
print ("prec matrix",preck)
print ("mprec matrix",mpreck)

prec@1: 0.454 prec@3: 0.669 prec@5: 0.765
mprec@1: 1.509e-02 mprec@3: 6.147e-02 mprec@5: 1.019e-01
prec matrix [0.45363636 0.59242424 0.66939394 0.71393939 0.76454545]
mprec matrix [0.01508809 0.04888622 0.06147312 0.08406356 0.1019489 ]


In [ ]:
# 2 
# same model diff train file

In [87]:
tr_latent_X = data_utils.TensorDataset(train_set_X, train_set_y)
te_latent_X = data_utils.TensorDataset(test_set_X,  test_set_y)
train_loader_X = torch.utils.data.DataLoader(dataset=tr_latent_X)
test_loader_X = torch.utils.data.DataLoader(dataset=te_latent_X)

In [88]:
def train(epoch):
    model.train()

    train_loss = 0

    for batch_idx, (data,label) in enumerate(train_loader_X):

        model_opt.zero_grad()

        out = model(data)
        loss = loss_function(out, label)

        loss.backward()
        train_loss += loss.item()
        model_opt.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader_X.dataset),
                100. * batch_idx / len(train_loader_X), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader_X.dataset)))

In [89]:
# 2
for epoch in range(1, 10):
    train(epoch)

Train Epoch: 1 [0/6700 (0%)]	Loss: 2.049522
Train Epoch: 1 [100/6700 (1%)]	Loss: 0.885262
Train Epoch: 1 [200/6700 (3%)]	Loss: 0.256305
Train Epoch: 1 [300/6700 (4%)]	Loss: 5.151550
Train Epoch: 1 [400/6700 (6%)]	Loss: 5.028206
Train Epoch: 1 [500/6700 (7%)]	Loss: 0.602708
Train Epoch: 1 [600/6700 (9%)]	Loss: 1.652547
Train Epoch: 1 [700/6700 (10%)]	Loss: 0.907288
Train Epoch: 1 [800/6700 (12%)]	Loss: 1.879640
Train Epoch: 1 [900/6700 (13%)]	Loss: 0.296418
Train Epoch: 1 [1000/6700 (15%)]	Loss: 3.316686
Train Epoch: 1 [1100/6700 (16%)]	Loss: 0.000075
Train Epoch: 1 [1200/6700 (18%)]	Loss: 20.039415
Train Epoch: 1 [1300/6700 (19%)]	Loss: 0.105142
Train Epoch: 1 [1400/6700 (21%)]	Loss: 0.421650
Train Epoch: 1 [1500/6700 (22%)]	Loss: 0.378309
Train Epoch: 1 [1600/6700 (24%)]	Loss: 4.647684
Train Epoch: 1 [1700/6700 (25%)]	Loss: 0.019948
Train Epoch: 1 [1800/6700 (27%)]	Loss: 1.970050
Train Epoch: 1 [1900/6700 (28%)]	Loss: 1.825400
Train Epoch: 1 [2000/6700 (30%)]	Loss: 4.814600
Train Epoc

Train Epoch: 3 [3700/6700 (55%)]	Loss: 0.082510
Train Epoch: 3 [3800/6700 (57%)]	Loss: 0.069798
Train Epoch: 3 [3900/6700 (58%)]	Loss: 0.033343
Train Epoch: 3 [4000/6700 (60%)]	Loss: 2.509884
Train Epoch: 3 [4100/6700 (61%)]	Loss: 0.322819
Train Epoch: 3 [4200/6700 (63%)]	Loss: 0.003722
Train Epoch: 3 [4300/6700 (64%)]	Loss: 1.529987
Train Epoch: 3 [4400/6700 (66%)]	Loss: 0.006075
Train Epoch: 3 [4500/6700 (67%)]	Loss: 1.434485
Train Epoch: 3 [4600/6700 (69%)]	Loss: 1.347577
Train Epoch: 3 [4700/6700 (70%)]	Loss: 0.170593
Train Epoch: 3 [4800/6700 (72%)]	Loss: 2.673012
Train Epoch: 3 [4900/6700 (73%)]	Loss: 0.080038
Train Epoch: 3 [5000/6700 (75%)]	Loss: 1.100973
Train Epoch: 3 [5100/6700 (76%)]	Loss: 0.099057
Train Epoch: 3 [5200/6700 (78%)]	Loss: 0.706849
Train Epoch: 3 [5300/6700 (79%)]	Loss: 0.366289
Train Epoch: 3 [5400/6700 (81%)]	Loss: 0.191033
Train Epoch: 3 [5500/6700 (82%)]	Loss: 3.179729
Train Epoch: 3 [5600/6700 (84%)]	Loss: 2.778670
Train Epoch: 3 [5700/6700 (85%)]	Loss: 2

Train Epoch: 6 [600/6700 (9%)]	Loss: 0.075927
Train Epoch: 6 [700/6700 (10%)]	Loss: 0.565931
Train Epoch: 6 [800/6700 (12%)]	Loss: 1.058079
Train Epoch: 6 [900/6700 (13%)]	Loss: 0.084677
Train Epoch: 6 [1000/6700 (15%)]	Loss: 0.084108
Train Epoch: 6 [1100/6700 (16%)]	Loss: 0.000036
Train Epoch: 6 [1200/6700 (18%)]	Loss: 0.000903
Train Epoch: 6 [1300/6700 (19%)]	Loss: 0.013028
Train Epoch: 6 [1400/6700 (21%)]	Loss: 0.005875
Train Epoch: 6 [1500/6700 (22%)]	Loss: 0.070695
Train Epoch: 6 [1600/6700 (24%)]	Loss: 0.000004
Train Epoch: 6 [1700/6700 (25%)]	Loss: 0.000816
Train Epoch: 6 [1800/6700 (27%)]	Loss: 0.000361
Train Epoch: 6 [1900/6700 (28%)]	Loss: 0.490602
Train Epoch: 6 [2000/6700 (30%)]	Loss: 6.388147
Train Epoch: 6 [2100/6700 (31%)]	Loss: 0.026410
Train Epoch: 6 [2200/6700 (33%)]	Loss: 0.193434
Train Epoch: 6 [2300/6700 (34%)]	Loss: 0.001366
Train Epoch: 6 [2400/6700 (36%)]	Loss: 0.083673
Train Epoch: 6 [2500/6700 (37%)]	Loss: 0.015256
Train Epoch: 6 [2600/6700 (39%)]	Loss: 0.1391

Train Epoch: 8 [4300/6700 (64%)]	Loss: 0.018532
Train Epoch: 8 [4400/6700 (66%)]	Loss: 0.000006
Train Epoch: 8 [4500/6700 (67%)]	Loss: 0.335584
Train Epoch: 8 [4600/6700 (69%)]	Loss: 0.350255
Train Epoch: 8 [4700/6700 (70%)]	Loss: 0.018560
Train Epoch: 8 [4800/6700 (72%)]	Loss: 2.933100
Train Epoch: 8 [4900/6700 (73%)]	Loss: 0.008084
Train Epoch: 8 [5000/6700 (75%)]	Loss: 0.385090
Train Epoch: 8 [5100/6700 (76%)]	Loss: 0.007060
Train Epoch: 8 [5200/6700 (78%)]	Loss: 0.161712
Train Epoch: 8 [5300/6700 (79%)]	Loss: 0.240506
Train Epoch: 8 [5400/6700 (81%)]	Loss: 0.000093
Train Epoch: 8 [5500/6700 (82%)]	Loss: 0.372610
Train Epoch: 8 [5600/6700 (84%)]	Loss: 0.900231
Train Epoch: 8 [5700/6700 (85%)]	Loss: 2.750024
Train Epoch: 8 [5800/6700 (87%)]	Loss: 0.698008
Train Epoch: 8 [5900/6700 (88%)]	Loss: 0.323333
Train Epoch: 8 [6000/6700 (90%)]	Loss: 0.028429
Train Epoch: 8 [6100/6700 (91%)]	Loss: 0.000160
Train Epoch: 8 [6200/6700 (93%)]	Loss: 0.019020
Train Epoch: 8 [6300/6700 (94%)]	Loss: 0

In [79]:
ans = model.predict(test_set_X)
k = 5
y_pr = ans.numpy() 
y_pred = np.argsort(-y_pr,axis=1)[:,:k]
print (y_pred)
print (y_test)

[[ 3 12 22  4  7]
 [ 2  4  1  9 21]
 [ 3 12  4  1 39]
 ...
 [ 9  2 45 15 16]
 [ 4 37  5 18  2]
 [ 3 12 37 22 10]]
[3. 2. 3. ... 9. 4. 3.]


/home/aditya/Vpy35/lib/python3.5/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [80]:
# eval functions for Deep Learning
preck = utils.getPrecAtK( y_test, y_pred, k )
# The macro precision code takes a bit longer to execute due to the for loop over labels
mpreck = utils.getMPrecAtK( y_test, y_pred, k )

# According to our definitions, both prec@k and mprec@k should go up as k goes up i.e. for your
# method, prec@i > prec@j if i > j and mprec@i > mprec@j if i > j. See the assignment description
# to convince yourself why this must be the case.

print( "prec@1: %0.3f" % preck[0], "prec@3: %0.3f" % preck[2], "prec@5: %0.3f" % preck[4] )
# Dont be surprised if mprec is small -- it is hard to do well on rare error classes
print( "mprec@1: %0.3e" % mpreck[0], "mprec@3: %0.3e" % mpreck[2], "mprec@5: %0.3e" % mpreck[4] )
print ("prec matrix",preck)
print ("mprec matrix",mpreck)

prec@1: 0.762 prec@3: 0.918 prec@5: 0.955
mprec@1: 8.627e-03 mprec@3: 6.489e-02 mprec@5: 1.113e-01
prec matrix [0.76212121 0.88       0.91757576 0.94030303 0.95545455]
mprec matrix [0.00862691 0.0407148  0.06489299 0.09325235 0.11130134]
